# SecML Malware Tutorial

In this tutorial, you will learn how to use this plugin to test the already implemented attacks against a PyTorch network of your choice.

In [1]:
import os
import magic
from secml.array import CArray

from secml_malware.models.malconv import MalConv
from secml_malware.models.c_classifier_end2end_malware import CClassifierEnd2EndMalware, End2EndModel

net = MalConv() # CNN trained on RAW BYTES
net = CClassifierEnd2EndMalware(net)
net.load_pretrained_model()

Firstly, we have created the network (MalConv) and it has been passed wrapped with a *CClassifierEnd2EndMalware* model class.
This object generalizes PyTorch end-to-end ML models.
Since MalConv is already coded inside the plugin, the weights are also stored, and they can be retrieved with the *load_pretrained_model* method.

If you wish to use diffierent weights, pass the path to the PyTorch *pth* file to that method.

In [2]:
from secml_malware.attack.whitebox.c_header_evasion import CHeaderEvasion

partial_dos = CHeaderEvasion(net, random_init=False, iterations=50, optimize_all_dos=False, threshold=0.0)

This is how an attack is created, no further action is needed.
The `random_init` parameter specifies if the bytes should be assigned with random values before beginning the optimization process, `iterations` sets the number of steps of the attack, `optimize_all_dos` sets if all the DOS header should be perturbed, or just the first 58 bytes, while `threshold` is the detection threshold used as a stopping condition.

If you want to see how much the network is deteriorated by the attack, set this parameter to 0, or it will stop as soon as the confidence decreases below such value.

In [3]:
folder = "secml_malware/data/malware_samples/test_folder"
X = []
y = []
file_names = []
for i, f in enumerate(os.listdir(folder)):
    path = os.path.join(folder, f)
    if 'petya' not in path:
        continue
    if "PE32" not in magic.from_file(path):
        continue
    with open(path, "rb") as file_handle:
        code = file_handle.read()
    x = End2EndModel.bytes_to_numpy(
        code, net.get_input_max_length(), 256, False
    )
    _, confidence = net.predict(CArray(x), True)

    if confidence[0, 1].item() < 0.5:
        continue

    print(f"> Added {f} with confidence {confidence[0,1].item()}")
    X.append(x)
    conf = confidence[1][0].item()
    y.append([1 - conf, conf])
    file_names.append(path)
    # PETYA RANSOMWARE
# 0.9112256765365601

> Added petya.file with confidence 0.9112256765365601


We load a simple dataset from the `malware_samples/test_folder` that you have filled with malware to test the attacks.
We discard all the samples that are not seen by the network.
The `CArray` class is the base object you will handle when dealing with vectors in this library.

In [4]:
for sample, label in zip(X, y):
    y_pred, adv_score, adv_ds, f_obj = partial_dos.run(CArray(sample), CArray(label[1]))
    print(partial_dos.confidences_)
    print(f_obj)

[0.9112256765365601, 0.06050168350338936, 0.02569282241165638, 0.013263005763292313, 0.007708318531513214, 0.006512514315545559, 0.00256469938904047, 0.0010069687850773335, 0.00042997609125450253, 0.00031924922950565815, 0.0005989515921100974, 0.0002068594767479226, 0.00011205733608221635, 8.947109017753974e-05, 8.69965588208288e-05, 0.00044750436791218817, 0.00020894983026664704, 0.00012765263090841472, 0.00010658507380867377, 0.0003574001893866807, 0.00016230311302933842, 0.00010355764243286103, 8.281883492600173e-05, 7.962167001096532e-05, 0.0005361491348594427, 0.00022481686028186232, 0.0001238513650605455, 9.08773290575482e-05, 7.962690142448992e-05, 7.131584425223991e-05, 7.284294406417757e-05, 0.00020154037338215858, 0.00037678476655855775, 0.0001889022532850504, 0.00011984687444055453, 0.00010659441613825038, 0.00022499141050502658, 0.0003707527357619256, 0.00016528736159671098, 0.0001126341157942079, 8.412790339207277e-05, 7.938232738524675e-05, 7.359428855124861e-05, 7.929849

Inside the `adv_ds` object, you can find the adversarial example computed by the attack.
You can reconstruct the functioning example by using a specific function inside the plugin:

In [5]:
adv_x = adv_ds.X[0,:]
real_adv_x = partial_dos.create_real_sample_from_adv(file_names[0], adv_x)
print(len(real_adv_x))
real_x = End2EndModel.bytes_to_numpy(real_adv_x, net.get_input_max_length(), 256, False)
_, confidence = net.predict(CArray(real_x), True)
print(confidence[0,1].item())

806912
0.00015971656830515712


... and you're done!
If you want to create a real sample (stored on disk), just have a look at the `create_real_sample_from_adv` of each attack. It accepts a third string argument that will be used as a destination file path for storing the adversarial example.

## Bonus: more attacks!
We used one attack, which is the Partial DOS one. But what if we want to use others?
Easy peasy task! Just open the [source code](https://github.com/pralab/secml_malware/tree/master/secml_malware/attack/whitebox) or the [documentation](https://secml-malware.readthedocs.io/en/docs/source/secml_malware.attack.whitebox.html) of the other white box attacks, and instantiate the one you like!
Let's use the [FGSM attack](https://arxiv.org/abs/1802.04528), for instance:

In [ ]:
from secml_malware.attack.whitebox import CKreukEvasion

fgsm = CKreukEvasion(net, how_many_padding_bytes=1024, epsilon=4.0, iterations=5)
for i, (sample, label) in enumerate(zip(X, y)):
    y_pred, adv_score, adv_ds, f_obj = fgsm.run(CArray(sample), CArray(label[1]))
    print(fgsm.confidences_)
    print(f_obj)
    real_adv_x = fgsm.create_real_sample_from_adv(file_names[i], adv_ds.X[i, :])
    with open(file_names[i], 'rb') as f:
        print('Original length: ', len(f.read()))
    print('Adversarial sample length: ', len(real_adv_x))


... and you're done! Remember that this particular attack might take a while, depending on how many bytes the algorithm is tasked to edit (and also for the number of iterations).
In the meantime, **happy coding with SecML Malware!**